# How to use regioinvent

Note that running this entire notebook will take you around 2 to 3 hours if you choose the highest cutoff option.

To be able to use regioinvent, you will need:
- to install the brightway2 Python library (brightway2 and NOT brightway2.5), easier is to get it through activity-browser: https://github.com/LCA-ActivityBrowser/activity-browser
- a brightway project within which there is an ecoinvent database with either the version 3.9/3.9.1/3.10/3.10.1 cutoff
- to download the latest version of the trade database stored here: https://doi.org/10.5281/zenodo.11583814

In [ ]:
import pandas as pd
import numpy as np
import sys
# change the path here to wherever you stored the Regioinvent Python package
sys.path.append('C://Users/11max/PycharmProjects/Regioinvent/')
import regioinvent

The initialization of the Regioinvent class requires 3 arguments:
- the name of the brightway2 project where you stored ecoinvent and where regioinvent will be added
- the name you gave your ecoinvent database within your brightway2 project
- the version of the ecoinvent database ('3.9', '3.9.1', '3.10' or '3.10.1')

In [ ]:
regio = regioinvent.Regioinvent(bw_project_name='ecoinvent3.9.1', ecoinvent_database_name='ecoinvent-3.9.1-cutoff', ecoinvent_version='3.9.1')

First step of regioinvent is to spatialize the original ecoinvent database. This entails two steps:
- Creating a new biosphere database for spatialized elementary flows (e.g., Ammonia, CA-QC)
- Spatializing the elementary flows used within the ecoinvent database, based on the location of the process itself

In [ ]:
regio.spatialize_my_ecoinvent()

This creates two databases in your brightway2 project:
- "_biosphere3_spatialized_flows_", which contains all the newly created spatialized elementary flows
- "the-name-of-your-ecoinvent-database regionalized", which is a copy of the original ecoinvent but using spatialized elementary flows
You thus end up with two ecoinvent versions: the original one and a spatialized one.

Because elementary flows are now spatialized, you will need a specific LCIA method to operate with any flow (spatialized or not). The following function imports such methods. Currently available: "IW v2.1", "EF v3.1", "ReCiPe 2016 v1.03 (H)". Can also import all of them in one go.

In [ ]:
regio.import_fully_regionalized_impact_method(lcia_method='all')

If you want to go further in the regionalization, i.e., to create new national production processes and to rely on trade data to create regionalized consumption markets of ecoinvent, you can run the next function. There are 3 arguments:
- _trade_database_path_ which is the path where you stored the trade database you downloaded from Zenodo: https://doi.org/10.5281/zenodo.11583814
- _regioinvent_database_name_ which is simply the name that the created database will take
- _cutoff_ which is the cut-off (between 0 and 1) beyond which countries will be aggreated as RoW. For more info on what cutoff does, see section "Selection of countries for regionalization" of the Methodology.md file.

In [ ]:
regio.regionalize_ecoinvent_with_trade(trade_database_path='C://Users/11max/PycharmProjects/Regioinvent/trade_data.db',
                                       regioinvent_database_name='Regioinvent',
                                       cutoff=0.99)

This automatically wrote the regioinvent database in your brightway project. So you can now go on brightway2 or AB to perform your LCAs normally with regioinvent.